In [1]:
import requests,json
import os
import csv
import pyodbc
import pandas as pd

In [61]:
os.getcwd()

'G:\\My Drive\\Business\\Fractal\\project-imagineers-sep-2021'

In [2]:
cities = ['New York',
'Austin',
'San Francisco',
'Atlanta',
'Seattle',
'Boston',
'Los Angeles',
'Houston',
'Chicago',
'Miami']

## Reading data from weather API and save the data in CSV

In [4]:
# importing requests and json
import requests, json
# base URL
BASE_URL = "https://api.openweathermap.org/data/2.5/weather?"
# API key 
API_KEY = "1b1d68528c0943235c89c1aa1dd1e0b6"
# upadting the URL
row = []

with open('./city_weather.csv','w',newline = '\n') as f :
    file_writer = csv.writer(f,delimiter = ',')
    file_writer.writerow(['city','humidity','pressure','temperature'])
    for city in cities :
        response = requests.get(BASE_URL + "q=" + city + "&appid=" + API_KEY)
        row.append(city)
        data = response.json()
        row.append(data['main']['humidity'])
        row.append(data['main']['pressure'])
        row.append(data['main']['temp'])
        file_writer.writerow(row)
        row = []
    f.close()     
        
    

## Reading data from an API and save it in a table in Database

In [5]:
server = 'mskl-db-server.database.windows.net'
database = 'fractal-db'
username = 'learner01'
password = '!@#sw2aq1'
driver= '{ODBC Driver 17 for SQL Server}'

In [6]:
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [79]:
# importing requests and json
import requests, json
# base URL
BASE_URL = "https://api.openweathermap.org/data/2.5/weather?"
# API key 
API_KEY = "1b1d68528c0943235c89c1aa1dd1e0b6"
# upadting the URL
row = []
df = pd.DataFrame(columns = ['city','humidity','pressure','temperature'])

for city in range(len(cities)) :
    response = requests.get(BASE_URL + "q=" + city + "&appid=" + API_KEY)
    data = response.json()
    cursor.execute("INSERT INTO city_temperatures(city_name,humidity,pressure,temperature) values(?,?,?,?)",
                   city,
                   data['main']['humidity'],
                   data['main']['pressure'],
                   data['main']['temp'])
    
cnxn.commit()
cursor.close()  

In [87]:
cursor.execute("""
               select * from city_temperatures where humidity = 
               (select min(humidity) from city_temperatures)
               """
              )

for row in cursor:
    print(row)

cnxn.commit()
cursor.close()

(18, 'Los Angeles', 75.0, 1015.0, 294.1)


### Question : 

* Create a summary table of month-wise order statistics : (Use only Python) (Naming Convention : <your_name_initials>_order_summary)
* Columns : 
    * Year
    * Month
    * No. of Orders
    * Total Order Value
    * Avg. Basket Value
    * Avg. Order-to-Delivery Duration
    * Avg. Delay
    * No of Late Orders

* Use Python to perform data wrangling and prepare a dataframe having structure as shown above
* Write the dataframe back to the table
* Connect with SQL Server and run a query on the new table